In [ ]:
#Import Dependencies
import pandas as pd
import time
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests

In [ ]:
executable_path = {'executable_path': 'c:/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
#Mars news site to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)
html=browser.html
soup = bs(html, 'html.parser')

#retrieve most recent news title and paragraph
news_title = soup.find('div', class_='content_title').text
news_p = soup.find('div', class_='article_teaser_body').text

print(news_title)
print(news_p)